In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("src")

from decoder import TweetyBertClassifier
from spectogram_generator import WavtoSpec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folder = "/media/george-vengrovski/disk2/canary/unsorted/USA5206"
classifier_path = "/media/george-vengrovski/disk1/linear_decoder_test"

dst_folder = "/media/george-vengrovski/disk2/canary/unsorted/USA5206_specs"
os.makedirs(dst_folder, exist_ok=True)

wav_to_spec = WavtoSpec(folder, dst_folder, csv_file_dir=None)

for day in os.listdir(folder):
    day_path = os.path.join(folder, day)
    if os.path.isdir(day_path):
        for song in os.listdir(day_path):
            file_path = os.path.join(day_path, song)
            if file_path.lower().endswith('.wav'):
                try:
                    WavtoSpec.process_file(wav_to_spec, file_path, None)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

print("Processing complete.")